In [ ]:
# Importing required libraries
import pandas as pd
import re
import string
import nltk
import numpy as np
import time
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [ ]:
#Load dataset
df = pd.read_csv('/content/drive/MyDrive/Placement Projects/Chat Bot/AMSC1 (1).csv')

In [ ]:
# Function to add new entries to the DataFrame
def add_entry(df, question, category, answer):
    # Create a new DataFrame row
    new_entry = pd.DataFrame({
        "Question": [question],
        "Category": [category],
        "Answer": [answer]
    })

    # Append the new entry to the existing DataFrame
    df = pd.concat([df, new_entry], ignore_index=True)

    return df

In [ ]:
df["Category"].unique()

array(['Course Overview', 'Curriculum and Subjects',
       'Admission and Cutoffs', 'Placement Opportunities', 'Internships',
       'Student Life', 'Future Prospects', 'Department and Faculty',
       'Miscellaneous', 'PhD', 'research'], dtype=object)

In [ ]:
# #question = input("Enter the question: ")
# category = "Department and Faculty"
# answer =  "The M.Tech program in Applied Mathematics and Scientific Computing is designed to equip students with advanced mathematical and computational skills. It emphasizes mathematical modeling, numerical methods, and optimization techniques, providing hands-on experience in a dedicated lab with high-configuration computers. The program encourages interdisciplinary research and collaboration with experienced faculty members to solve real-world problems."
# question = ["What details can you provide about the M.Tech program?"
# "Can you describe the M.Tech course offered by the department?"
# "What is the focus of the M.Tech program in Applied Mathematics and Scientific Computing?"
# "How does the M.Tech program prepare students for their careers?"
# "Could you elaborate on what students can expect from the M.Tech program?"

#             ]
# # Add the new entry to the DataFrame
# for i in question:
#   df = add_entry(df, i, category, answer)

In [ ]:
# df.to_csv("AMSC1.csv")

In [ ]:
df

,Unnamed: 0,Category,Question,Answer,N_Question,vector
0,0,Course Overview,What does the M.Tech in Applied Mathematics an...,It's an interdisciplinary program that integra...,mlech appli mathemat scientif comput encompass,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ..."
1,1,Course Overview,Can you provide details about the M.Tech progr...,This program combines mathematics and computat...,provid detail mlech program appli mathemat sci...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ..."
2,2,Course Overview,What topics are included in the M.Tech in Appl...,The curriculum covers advanced mathematical th...,topic includ mlech appli mathemat scientif comput,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ..."
3,3,Course Overview,What are the standout features of the Applied ...,The course emphasizes an integration of applie...,standout featur appli mathemat scientif comput...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ..."
4,4,Course Overview,Could you elaborate on the focus areas of the ...,It primarily focuses on the application of mat...,would elabor focus area appli mathemat scienti...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, ..."
...,...,...,...,...,...,...
251,251,Department and Faculty,What can you share regarding the department's ...,The Department of Applied Mathematics and Scie...,NaN,NaN
252,252,Department and Faculty,Could you explain what the Department of Appli...,The Department of Applied Mathematics and Scie...,NaN,NaN
253,253,Department and Faculty,What is the main objective of the department?,The Department of Applied Mathematics and Scie...,NaN,NaN
254,254,Department and Faculty,How does the department contribute to mathemat...,The Department of Applied Mathematics and Scie...,NaN,NaN


In [ ]:
# Download NLTK data (if necessary)
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
# Apostrophe expansion dictionary
Apostrophes_expansion = {
    "'s": " is",
    "'re": " are",
    "'r": " are",
    "n't": " not",
    "'ve": " have",
    "'d": " would",
    "'ll": " will",
    "dept.": "department"
}

In [ ]:
# Function to expand apostrophes
def remove_apostrophes(text):
    """Expand common apostrophes."""
    words = word_tokenize(text)
    processed_text = [Apostrophes_expansion[word] if word in Apostrophes_expansion else word for word in words]
    return " ".join(processed_text)

In [ ]:
# Stopwords removal
stop_words = stopwords.words('english')

def sw_rem(text):
    """Remove stopwords from text."""
    return ' '.join(word for word in text.split() if word not in stop_words)

In [ ]:
from textblob import TextBlob  # For spelling correction
# Spelling correction using TextBlob
def correct_spelling(text):
    """Correct spelling in the text using TextBlob."""
    corrected_text = str(TextBlob(text).correct())
    return corrected_text

In [ ]:
# Stemming
stems = SnowballStemmer('english')

def stemming(text):
    """Apply stemming to words in text."""
    return ' '.join(stems.stem(word) for word in text.split())

In [ ]:
# Text cleaning pipeline
def pipeline(text):
    """Apply text preprocessing steps: spelling correction, lowercase, punctuation removal, apostrophe expansion, stopwords removal, and stemming."""
    text = remove_apostrophes(text)  # Expand apostrophes
    text = correct_spelling(text)  # Correct spelling
    text = str(text).lower()  # Convert to lowercase
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)  # Remove punctuation

    text = sw_rem(text)  # Remove stopwords
    text = stemming(text)  # Apply stemming
    return text

In [ ]:
# Apply preprocessing to the questions in the dataset
df['N_Question'] = df['Question'].apply(pipeline)

In [ ]:
# Separate features and labels
x = df['N_Question']
y = df['Category']


In [ ]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=1)


In [ ]:
# Encode labels
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)

In [ ]:
# Vectorization using CountVectorizer
cv = CountVectorizer(max_features=500, stop_words='english')
X_train_v = cv.fit_transform(X_train).toarray()
X_test_v = cv.transform(X_test).toarray()

In [ ]:
# Train the model
gb = GradientBoostingClassifier()
gb.fit(X_train_v, y_train)

GradientBoostingClassifier()

In [ ]:
# Vectorization of the new input text
def vectorization(text):
    vector = cv.transform([text]).toarray()
    return list(vector[0])

# Add vector column to the dataset
df['vector'] = df['N_Question'].apply(vectorization)

In [ ]:
# Query function to retrieve the answer based on input
def quary(input_text):
    input_text = pipeline(input_text)
    input_vector = cv.transform([input_text]).toarray()

    # Predict the category for the input text
    input_category = list(label_encoder.inverse_transform([gb.predict(input_vector)[0]]))[0]

    # Filter dataset for matching category
    df_cat = df[df['Category'] == input_category]

    # Get the similarity of the input with the available questions
    all_vec = np.array(df_cat['vector'].to_list())
    text_similarity = cosine_similarity(X=all_vec, Y=input_vector)

    # Sort and retrieve the top 3 most similar answers
    answers = sorted(list(enumerate(text_similarity)), reverse=True, key=lambda x: x[1])

    # Check for similarity threshold
    if answers[0][1] <= 0.5:
        print("I don't have that information. Could you clarify or rephrase?")
        return

    # Output the best match and ask for feedback
    answer = df_cat.iloc[answers[0][0]]['Answer']
    print(f"Bot: {answer}")
    print()


In [ ]:


# Main chatbot function
def bot():
    """Run the chatbot in an interactive loop."""
    # Greeting and name input
    greetings = ["Hi there!", "Hello!"]
    goodbyes = ["Goodbye!", "Take care!", "Have a great day ahead!"]

    name = input("What should I call you? ")
    print(f"{np.random.choice(greetings)} How can I help you, {name}?")
    print()

    while True:
        # Take user input
        input_text = input("User: ")

        # Exit condition
        if input_text.lower() in ['bye', 'goodbye', 'exit']:
            print(f"Bot: {np.random.choice(goodbyes)}")
            break

        time.sleep(1)

        # Respond based on query
        quary(input_text)
        print()

        time.sleep(1)


In [ ]:
# Run the chatbot
bot()

What should I call you? apurba
Hey! How can I assist you today? How can I help you, apurba?

User: how many students are in mtech?
Bot: The department currently has approximately 200 students enrolled across various programs.


User: what about mtech?
I don't have that information. Could you clarify or rephrase?

User: bye
Bot: Take care!
